In [1]:
import os
import requests
from folium import Map, TileLayer

In [2]:
def submit_request(BASE_URL, endpoint, url):
    r = requests.get(
        f"{BASE_URL}/{endpoint}",
        params = {
            "url": url,
        }
    ).json()
    return r

In [3]:
# Add the GeoTIFF overlay
titiler_endpoint = "http://127.0.0.1:8000" 

# the tif cloud storage url
# incase the user does not select any map, use the default map
tif_url = 'http://localhost:8000/out_2030/lumap.tiff'


# instantiate the map
map = Map(location=[-24.162,132.847], 
         zoom_start=5)

# base map layer
base_layer = TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(map)

base_layer.add_to(map)

In [6]:
# get tile data from titiler
tile_json_analytic = submit_request(titiler_endpoint, "cog/tilejson.json", tif_url)
tileset = tile_json_analytic["tiles"][0]

KeyError: 'tiles'

In [5]:
tile_json_analytic

{'detail': 'HTTP response code: 404'}

In [ ]:



tile_layer = TileLayer(
    tiles= tileset,
    attr="LUTO Land Use Map",
)

tile_layer.add_to(map)